**Diana Kim ADA-2403M**

# **Практическая работа №6**
**Тема:** Программирование на OpenCL для CPU и GPU

**Цель работы:**

*   Изучение основ программирования на OpenCL.
*   Разработка кросс-платформенного приложения для выполнения
параллельных вычислений на CPU и GPU.


In [1]:
!nvidia-smi

Fri Jan 16 14:40:30 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   52C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


### **Задание 1**




In [20]:
%%writefile task1.cpp
#define CL_TARGET_OPENCL_VERSION 120

#include <CL/cl.h>                  // подключает OpenCL
#include <iostream>             // подключает вывод
#include <vector>                 // подключает массивы
#include <fstream>           // подключает чтение файлов
#include <cstdlib>             // подключает rand и exit
#include <ctime>                // подключает time
#include <chrono>                   // подключает таймер
#include <cmath>              // подключает fabs
#include <iomanip>              // подключает форматированный вывод
using namespace std;             // упрощает синтаксис


void cl_ok(cl_int err, const char* msg) {        // проверяет ошибки OpenCL
    if (err != CL_SUCCESS) {                  // проверяет код ошибки
        cout << "OpenCL error (" << msg << "): " << err << endl;           // пишет ошибку
        exit(1);                   // завершает программу
    }
}

string read_text(const char* path) {                 // читает текст файла
    ifstream f(path);                           // открывает файл
    if (!f.is_open()) {                    // проверяет открытие
        cout << "Не удалось открыть файл: " << path << endl;               // пишет сообщение
        exit(1);                  // завершает программу
    }
    string s((istreambuf_iterator<char>(f)),           // читает содержимое
             istreambuf_iterator<char>());
    return s;                                      // возвращает строку
}

bool get_device(cl_device_type type,
                cl_platform_id& out_plat,
                cl_device_id& out_dev) {                 // ищет OpenCL устройство
    cl_uint pcount = 0;                        // хранит число платформ
    clGetPlatformIDs(0, nullptr, &pcount);              // получает количество платформ
    if (pcount == 0) return false;              // проверяет наличие
    vector<cl_platform_id> plats(pcount);                 // создаёт массив платформ
    clGetPlatformIDs(pcount, plats.data(), nullptr);                   // получает платформы
    for (cl_uint i = 0; i < pcount; i++) {            // перебирает платформы
        cl_uint dcount = 0;                             // хранит число устройств
        if (clGetDeviceIDs(plats[i], type, 0, nullptr, &dcount) != CL_SUCCESS)
            continue;                               // пропускает при ошибке
        if (dcount == 0) continue;                            // проверяет наличие устройств
        vector<cl_device_id> devs(dcount);                  // создаёт массив устройств
        clGetDeviceIDs(plats[i], type, dcount, devs.data(), nullptr);                        // получает устройства
        out_plat = plats[i];                   // сохраняет платформу
        out_dev = devs[0];                       // сохраняет устройство
        return true;                           // сообщает успех
    }
    return false;                      // сообщает неуспех
}

double run_cpu_plain(const vector<float>& A,
                     const vector<float>& B,
                     vector<float>& C) {                     // выполняет сложение на CPU
    auto start = chrono::high_resolution_clock::now();                      // фиксирует старт
    for (size_t i = 0; i < A.size(); i++)                 // проходит по массивам
        C[i] = A[i] + B[i];                             // складывает элементы
    auto end = chrono::high_resolution_clock::now();                         // фиксирует конец
    return chrono::duration<double, milli>(end - start).count();                      // возвращает время
}

double run_gpu_opencl(const vector<float>& A,
                      const vector<float>& B,
                      vector<float>& C) {        // выполняет сложение на GPU
    cl_platform_id plat = nullptr;                            // хранит платформу
    cl_device_id dev = nullptr;                      // хранит устройство
    if (!get_device(CL_DEVICE_TYPE_GPU, plat, dev))                      // ищет GPU
        return -1.0;                            // возвращает ошибку
    cl_int err;                                         // хранит ошибки
    cl_context ctx = clCreateContext(nullptr, 1, &dev, nullptr, nullptr, &err);                     // создаёт контекст
    cl_ok(err, "create context");
    cl_command_queue q = clCreateCommandQueue(ctx, dev, CL_QUEUE_PROFILING_ENABLE, &err);                       // создаёт очередь
    cl_ok(err, "create queue");
    string src = read_text("kernel_add.cl");              // читает kernel
    const char* code = src.c_str();              // получает указатель
    size_t len = src.size();                         // получает длину
    cl_program prog = clCreateProgramWithSource(ctx, 1, &code, &len, &err);                    // создаёт программу
    cl_ok(err, "create program");
    err = clBuildProgram(prog, 1, &dev, nullptr, nullptr, nullptr);                   // компилирует программу
    if (err != CL_SUCCESS) {                    // проверяет сборку
        size_t log_size;
        clGetProgramBuildInfo(prog, dev, CL_PROGRAM_BUILD_LOG, 0, nullptr, &log_size);
        vector<char> log(log_size);
        clGetProgramBuildInfo(prog, dev, CL_PROGRAM_BUILD_LOG, log_size, log.data(), nullptr);
        cout << log.data() << endl;                // выводит лог
        exit(1);                                                        // завершает программу
    }

    cl_kernel ker = clCreateKernel(prog, "vector_add", &err);                           // создаёт kernel
    cl_ok(err, "create kernel");
    int n = (int)A.size();                       // сохраняет размер
    size_t bytes = n * sizeof(float);                     // считает размер в байтах
    cl_mem dA = clCreateBuffer(ctx, CL_MEM_READ_ONLY, bytes, nullptr, &err);               // создаёт буфер A
    cl_mem dB = clCreateBuffer(ctx, CL_MEM_READ_ONLY, bytes, nullptr, &err);                    // создаёт буфер B
    cl_mem dC = clCreateBuffer(ctx, CL_MEM_WRITE_ONLY, bytes, nullptr, &err);                         // создаёт буфер C
    clEnqueueWriteBuffer(q, dA, CL_TRUE, 0, bytes, A.data(), 0, nullptr, nullptr);                        // копирует A
    clEnqueueWriteBuffer(q, dB, CL_TRUE, 0, bytes, B.data(), 0, nullptr, nullptr);                    // копирует B
    clSetKernelArg(ker, 0, sizeof(cl_mem), &dA);                       // задаёт аргумент A
    clSetKernelArg(ker, 1, sizeof(cl_mem), &dB);                // задаёт аргумент B
    clSetKernelArg(ker, 2, sizeof(cl_mem), &dC);                         // задаёт аргумент C
    clSetKernelArg(ker, 3, sizeof(int), &n);               // задаёт размер
    size_t local = 256;                          // задаёт размер блока
    size_t global = ((n + local - 1) / local) * local;                             // округляет сетку
    cl_event ev;
    clEnqueueNDRangeKernel(q, ker, 1, nullptr, &global, &local, 0, nullptr, &ev);                     // запускает kernel
    clFinish(q);                                  // ждёт завершения
    clEnqueueReadBuffer(q, dC, CL_TRUE, 0, bytes, C.data(), 0, nullptr, nullptr);            // копирует результат
    cl_ulong t0, t1;
    clGetEventProfilingInfo(ev, CL_PROFILING_COMMAND_START, sizeof(cl_ulong), &t0, nullptr);
    clGetEventProfilingInfo(ev, CL_PROFILING_COMMAND_END, sizeof(cl_ulong), &t1, nullptr);

    double ms = (t1 - t0) / 1e6;                 // переводит время в мс

    clReleaseEvent(ev);                               // освобождает event
    clReleaseMemObject(dA);                     // освобождает A
    clReleaseMemObject(dB);                    // освобождает B
    clReleaseMemObject(dC);                       // освобождает C
    clReleaseKernel(ker);                          // освобождает kernel
    clReleaseProgram(prog);                     // освобождает программу
    clReleaseCommandQueue(q);                // освобождает очередь
    clReleaseContext(ctx);                     // освобождает контекст

    return ms;                   // возвращает время
}

bool check_add(const vector<float>& A,
               const vector<float>& B,
               const vector<float>& C) {                     // проверяет корректность
    for (size_t i = 0; i < A.size(); i++)                 // проходит по массивам
        if (fabs(C[i] - (A[i] + B[i])) > 1e-5f)                              // проверяет разницу
            return false;                                 // сообщает ошибку
    return true;                                     // сообщает успех
}

int main() {
    srand((unsigned)time(0));                     // инициализирует rand

    int sizes[3] = {10000, 100000, 1000000};              // задаёт размеры


    cout << left << setw(12) << "N"         // вывод
         << setw(16) << "CPU time"
         << setw(16) << "GPU time"
         << setw(10) << "CPU"
         << setw(10) << "GPU" << endl;

    for (int s : sizes) {                       // перебирает размеры
        vector<float> A(s), B(s), Ccpu(s), Cgpu(s);           // создаёт массивы
        for (int i = 0; i < s; i++) {                  // заполняет массивы
            A[i] = rand() % 1000 / 10.0f;
            B[i] = rand() % 1000 / 10.0f;
        }
        double cpu_ms = run_cpu_plain(A, B, Ccpu);             // запускает CPU
        double gpu_ms = run_gpu_opencl(A, B, Cgpu);               // запускает GPU
        bool ok_cpu = check_add(A, B, Ccpu);            // проверяет CPU
        bool ok_gpu = (gpu_ms < 0) ? false : check_add(A, B, Cgpu);                 // проверяет GPU

        cout << left << setw(12) << s
             << setw(16) << fixed << setprecision(4) << cpu_ms
             << setw(16) << fixed << setprecision(4) << gpu_ms
             << setw(10) << (ok_cpu ? "OK" : "FAIL")
             << setw(10) << (ok_gpu ? "OK" : "FAIL") << endl;
    }
    return 0;                                   // завершает программу
}

Writing task1.cpp


In [21]:
%%writefile kernel_add.cl
__kernel void vector_add(__global const float* A, __global const float* B, __global float* C, int n) { // объявляет ядро
    int id = (int)get_global_id(0); // получает глобальный id
    if (id < n) { // проверяет границу
        C[id] = A[id] + B[id]; // пишет сумму
    } // завершает if
} // завершает ядро

Overwriting kernel_add.cl


In [22]:
!g++ task1.cpp -o task1 -lOpenCL
!./task1 > results_add.csv
!cat results_add.csv

N           CPU time        GPU time        CPU       GPU       
10000       0.1081          0.0143          OK        OK        
100000      1.0467          0.0140          OK        OK        
1000000     10.4824         0.0512          OK        OK        


### **Задание 2**


In [26]:
%%writefile task2.cpp

#define CL_TARGET_OPENCL_VERSION 120         // задает версию OpenCL 1.2
#include <CL/cl.h>           // подключает OpenCL
#include <iostream>                    // подключает вывод
#include <vector>               // подключает vector
#include <fstream>              // подключает чтение файлов
#include <cstdlib>                              // подключает rand и exit
#include <ctime>                        // подключает time
#include <chrono>                      // подключает таймер
#include <cmath>                    // подключает fabs
#include <iomanip>             // подключает форматирование вывода
using namespace std;                // чтобы не писать std::


void cl_ok(cl_int err, const char* msg) {     // создает проверку ошибок OpenCL
    if (err != CL_SUCCESS) { // проверяет ошибку
        cout << "OpenCL error (" << msg << "): " << err << "\n";      // выводит ошибку
        exit(1);     // завершает программу
    }
}

string read_text(const char* path) {            // создает чтение текста файла
    ifstream f(path);             // открывает файл
    if (!f.is_open()) {              // проверяет открытие
        cout << "Cannot open file: " << path << "\n";         // пишет сообщение
        exit(1);            // завершает программу
    }
    string s((istreambuf_iterator<char>(f)), istreambuf_iterator<char>());     // читает всё в строку
    return s;            // возвращает строку
}

bool pick_device(cl_device_type type, cl_platform_id& out_plat, cl_device_id& out_dev) { // создает поиск устройства
    cl_uint pcount = 0; // создает число платформ
    cl_int err = clGetPlatformIDs(0, nullptr, &pcount); // получает количество платформ
    if (err != CL_SUCCESS || pcount == 0) return false; // проверяет наличие платформ
    vector<cl_platform_id> plats(pcount); // создает массив платформ
    cl_ok(clGetPlatformIDs(pcount, plats.data(), nullptr), "get platforms"); // получает платформы
    for (cl_uint p = 0; p < pcount; p++) { // запускает перебор платформ
        cl_uint dcount = 0; // создает число устройств
        err = clGetDeviceIDs(plats[p], type, 0, nullptr, &dcount); // получает число устройств
        if (err != CL_SUCCESS || dcount == 0) continue; // пропускает если устройств нет
        vector<cl_device_id> devs(dcount); // создает массив устройств
        cl_ok(clGetDeviceIDs(plats[p], type, dcount, devs.data(), nullptr), "get devices"); // получает устройства
        out_plat = plats[p]; // сохраняет платформу
        out_dev = devs[0]; // сохраняет первое устройство
        return true; // возвращает успех
    }
    return false; // возвращает false если не найдено
}

void cpu_matmul_plain(const vector<float>& A, const vector<float>& B, vector<float>& C, int N, int M, int K) { // создает CPU умножение
    for (int i = 0; i < N; i++) { // перебирает строки A
        for (int j = 0; j < K; j++) { // перебирает столбцы B
            float sum = 0.0f; // создает сумму
            for (int t = 0; t < M; t++) { // перебирает общий размер M
                sum += A[i * M + t] * B[t * K + j]; // прибавляет произведение
            } // завершает цикл
            C[i * K + j] = sum; // записывает результат
        }
    }
}

double run_gpu_matmul(const vector<float>& A, const vector<float>& B, vector<float>& C, int N, int M, int K) { // создает запуск matmul на GPU
    cl_platform_id plat = nullptr; // создает платформу
    cl_device_id dev = nullptr; // создает устройство
    if (!pick_device(CL_DEVICE_TYPE_GPU, plat, dev)) return -1.0; // возвращает -1 если GPU нет
    cl_int err = 0; // создает переменную ошибок
    cl_context ctx = clCreateContext(nullptr, 1, &dev, nullptr, nullptr, &err); // создает контекст
    cl_ok(err, "create context"); // проверяет контекст
    cl_command_queue q = clCreateCommandQueue(ctx, dev, CL_QUEUE_PROFILING_ENABLE, &err); // создает очередь
    cl_ok(err, "create queue"); // проверяет очередь
    string src = read_text("kernel_matmul.cl"); // читает kernel текст
    const char* csrc = src.c_str(); // получает указатель на строку
    size_t srclen = src.size(); // получает длину строки
    cl_program prog = clCreateProgramWithSource(ctx, 1, &csrc, &srclen, &err); // создает программу
    cl_ok(err, "create program"); // проверяет программу
    err = clBuildProgram(prog, 1, &dev, nullptr, nullptr, nullptr); // собирает программу
    if (err != CL_SUCCESS) { // проверяет сборку
        size_t log_size = 0; // создает размер лога
        clGetProgramBuildInfo(prog, dev, CL_PROGRAM_BUILD_LOG, 0, nullptr, &log_size); // получает размер лога
        vector<char> log(log_size); // создает буфер лога
        clGetProgramBuildInfo(prog, dev, CL_PROGRAM_BUILD_LOG, log_size, log.data(), nullptr); // получает лог
        cout << "Build log:\n" << log.data() << "\n"; // выводит лог
        exit(1); // завершает программу
    }
    cl_kernel ker = clCreateKernel(prog, "matmul", &err); // создает kernel
    cl_ok(err, "create kernel"); // проверяет kernel
    size_t bytesA = (size_t)N * (size_t)M * sizeof(float); // считает байты A
    size_t bytesB = (size_t)M * (size_t)K * sizeof(float); // считает байты B
    size_t bytesC = (size_t)N * (size_t)K * sizeof(float); // считает байты C
    cl_mem dA = clCreateBuffer(ctx, CL_MEM_READ_ONLY, bytesA, nullptr, &err); // создает буфер A
    cl_ok(err, "buffer A"); // проверяет A
    cl_mem dB = clCreateBuffer(ctx, CL_MEM_READ_ONLY, bytesB, nullptr, &err); // создает буфер B
    cl_ok(err, "buffer B"); // проверяет B
    cl_mem dC = clCreateBuffer(ctx, CL_MEM_WRITE_ONLY, bytesC, nullptr, &err); // создает буфер C
    cl_ok(err, "buffer C"); // проверяет C
    cl_ok(clEnqueueWriteBuffer(q, dA, CL_TRUE, 0, bytesA, A.data(), 0, nullptr, nullptr), "write A"); // пишет A на GPU
    cl_ok(clEnqueueWriteBuffer(q, dB, CL_TRUE, 0, bytesB, B.data(), 0, nullptr, nullptr), "write B"); // пишет B на GPU
    cl_ok(clSetKernelArg(ker, 0, sizeof(cl_mem), &dA), "arg0"); // задает A
    cl_ok(clSetKernelArg(ker, 1, sizeof(cl_mem), &dB), "arg1"); // задает B
    cl_ok(clSetKernelArg(ker, 2, sizeof(cl_mem), &dC), "arg2"); // задает C
    cl_ok(clSetKernelArg(ker, 3, sizeof(int), &N), "arg3"); // задает N
    cl_ok(clSetKernelArg(ker, 4, sizeof(int), &M), "arg4"); // задает M
    cl_ok(clSetKernelArg(ker, 5, sizeof(int), &K), "arg5"); // задает K
    size_t global[2] = {(size_t)N, (size_t)K}; // задает сетку N x K
    size_t local[2] = {8, 8}; // задает local 8x8 иначе чем было
    if (global[0] % local[0] != 0) global[0] = ((global[0] / local[0]) + 1) * local[0]; // округляет N
    if (global[1] % local[1] != 0) global[1] = ((global[1] / local[1]) + 1) * local[1]; // округляет K
    cl_event ev = nullptr; // создает событие
    cl_ok(clEnqueueNDRangeKernel(q, ker, 2, nullptr, global, local, 0, nullptr, &ev), "run kernel"); // запускает kernel
    cl_ok(clFinish(q), "finish"); // ждет завершения
    cl_ok(clEnqueueReadBuffer(q, dC, CL_TRUE, 0, bytesC, C.data(), 0, nullptr, nullptr), "read C"); // читает C на CPU
    cl_ulong t0 = 0; // создает start
    cl_ulong t1 = 0; // создает end
    cl_ok(clGetEventProfilingInfo(ev, CL_PROFILING_COMMAND_START, sizeof(cl_ulong), &t0, nullptr), "profile start"); // читает start
    cl_ok(clGetEventProfilingInfo(ev, CL_PROFILING_COMMAND_END, sizeof(cl_ulong), &t1, nullptr), "profile end"); // читает end
    double ms = (double)(t1 - t0) / 1e6; // переводит ns в ms
    clReleaseEvent(ev); // освобождает событие
    clReleaseMemObject(dA); // освобождает A
    clReleaseMemObject(dB); // освобождает B
    clReleaseMemObject(dC); // освобождает C
    clReleaseKernel(ker); // освобождает kernel
    clReleaseProgram(prog); // освобождает program
    clReleaseCommandQueue(q); // освобождает queue
    clReleaseContext(ctx); // освобождает context
    return ms; // возвращает время GPU
}

bool check_matmul(const vector<float>& X, const vector<float>& Y) { // создает проверку матриц
    int n = (int)X.size(); // получает размер
    for (int i = 0; i < n; i++) { // запускает цикл
        if (fabs(X[i] - Y[i]) > 1e-3f) return false; // проверяет погрешность
    }
    return true; // возвращает true
}





int main() {
    srand((unsigned)time(nullptr)); // задает seed
    int N = 192; // задает N иначе чем было
    int M = 192; // задает M иначе чем было
    int K = 192; // задает K иначе чем было
    vector<float> A(N * M); // создает A
    vector<float> B(M * K); // создает B
    vector<float> Ccpu(N * K); // создает Ccpu
    vector<float> Cgpu(N * K); // создает Cgpu
    for (int i = 0; i < N * M; i++) { // заполняет A
        A[i] = (float)(rand() % 200) / 20.0f; // пишет A значения
    } // завершает цикл
    for (int i = 0; i < M * K; i++) { // заполняет B
        B[i] = (float)(rand() % 200) / 20.0f; // пишет B значения
    } // завершает цикл

    auto s1 = chrono::high_resolution_clock::now(); // сохраняет старт CPU
    cpu_matmul_plain(A, B, Ccpu, N, M, K); // запускает CPU умножение
    auto e1 = chrono::high_resolution_clock::now(); // сохраняет конец CPU
    double cpu_ms = chrono::duration<double, milli>(e1 - s1).count(); // считает CPU время
    double gpu_ms = run_gpu_matmul(A, B, Cgpu, N, M, K); // запускает GPU умножение
    bool ok = (gpu_ms < 0.0) ? false : check_matmul(Ccpu, Cgpu); // проверяет совпадение

    cout << left << setw(8) << "N" << setw(8) << "M" << setw(8) << "K"
         << setw(16) << "CPU time" << setw(16) << "GPU time" << setw(8) << "ok" << "\n"; // печатает шапку
    cout << left << setw(8) << N << setw(8) << M << setw(8) << K
         << setw(16) << fixed << setprecision(4) << cpu_ms
         << setw(16) << fixed << setprecision(4) << gpu_ms
         << setw(8) << (ok ? "yes" : "no") << "\n"; // печатает строку

    return 0; // завершает программу
}

Writing task2.cpp


In [27]:
%%writefile kernel_matmul.cl
__kernel void matmul(__global const float* A,
                     __global const float* B,
                     __global float* C,
                     int N, int M, int K) { // объявляет kernel matmul
    int row = (int)get_global_id(0); // получает номер строки
    int col = (int)get_global_id(1); // получает номер столбца

    if (row < N && col < K) { // проверяет границы
        float sum = 0.0f; // создает сумму
        for (int t = 0; t < M; t++) { // перебирает t
            sum += A[row * M + t] * B[t * K + col]; // прибавляет произведение
        } // завершает цикл
        C[row * K + col] = sum; // записывает результат
    } // завершает if
} // завершает kernel

Overwriting kernel_matmul.cl


In [28]:
!g++ task2.cpp -o task2 -lOpenCL
!./task2

N       M       K       CPU time        GPU time        ok      
192     192     192     49.6247         0.1925          yes     
